In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.api.models import Sequential
from keras.api.layers import Input, LSTM, Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import psycopg2

In [2]:
host = "localhost"
port = "5432"
dbname = "stock_database"
user = "postgres"
password = "password"

conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)

query = """
SELECT date, priceoflasttransaction, max, min, avgprice, percentchange, 
       quantity, turnoverbest, totalturnover, companyname
FROM currency_data
"""

df = pd.read_sql_query(query, conn)

conn.close()

C:\Users\Milos\AppData\Local\Temp\ipykernel_11108\2494099808.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [3]:
df[['min', 'max']] = df[['min', 'max']].fillna(0)

In [4]:
total_rows_per_company = df.groupby("companyname").size()
missing_rows_per_company = df.groupby("companyname").apply(lambda group: group.isna().any(axis=1).sum())
missing_percentage = (missing_rows_per_company / total_rows_per_company) * 100

missing_stats = pd.DataFrame({
    "TotalRows": total_rows_per_company,
    "MissingRows": missing_rows_per_company,
    "MissingPercentage": missing_percentage
})

missing_stats.to_csv('missing_stats.csv', index_label="companyname")
no_data_companies = missing_stats[missing_stats["MissingPercentage"] == 100].index.tolist()
print("Companies with 100% missing data:", no_data_companies)

df_filtered = df[~df["companyname"].isin(no_data_companies)].copy()

C:\Users\Milos\AppData\Local\Temp\ipykernel_11108\2512222549.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_rows_per_company = df.groupby("companyname").apply(lambda group: group.isna().any(axis=1).sum())


Companies with 100% missing data: ['OBPP', 'SPOL', 'TRDB', 'VFPM']


In [5]:
df_filtered.ffill(inplace=True)
print("Missing values after forward fill:\n", df_filtered.isnull().sum())

Missing values after forward fill:
 date                      0
priceoflasttransaction    0
max                       0
min                       0
avgprice                  0
percentchange             0
quantity                  0
turnoverbest              0
totalturnover             0
companyname               0
dtype: int64


In [6]:
df_filtered["timestamp"] = pd.to_datetime(df_filtered["date"])
df_filtered.set_index("timestamp", inplace=True, verify_integrity=False)

df_filtered = df_filtered.sort_values(['companyname', 'timestamp'], ascending=[True, True])

df_filtered = df_filtered.drop(columns=['date'], axis=1)

C:\Users\Milos\AppData\Local\Temp\ipykernel_11108\1131642081.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_filtered["timestamp"] = pd.to_datetime(df_filtered["date"])


In [7]:
def clean_numeric(x):
    if isinstance(x, str):
        x = x.replace('.', '').replace(',', '.')
    return x

numeric_columns = ['priceoflasttransaction', 'max', 'min', 'avgprice', 
                   'percentchange', 'quantity', 'turnoverbest', 'totalturnover']

for col in numeric_columns:
    df_filtered[col] = df_filtered[col].apply(clean_numeric).astype(float)

In [8]:
lag = 3

In [9]:
periods = range(lag, 0, -1)

In [10]:
companies = list(set(df['companyname'].unique()) - set(no_data_companies))

In [11]:
df_filtered.dropna(inplace=True)
df_filtered = df_filtered[['companyname', 'priceoflasttransaction']]

In [12]:
df_filtered

,companyname,priceoflasttransaction
timestamp,,
2014-01-02,ADIN,250.0
2014-01-03,ADIN,250.0
2014-01-08,ADIN,250.0
2014-01-09,ADIN,250.0
2014-01-10,ADIN,250.0
...,...,...
2024-11-26,ZUAS,2100.0
2024-11-27,ZUAS,2100.0
2024-11-28,ZUAS,2100.0


In [13]:
import os

os.makedirs("models", exist_ok=True)
os.makedirs("scalers", exist_ok=True)

In [ ]:
for company in companies:
    company_data = df_filtered[df_filtered['companyname'] == company].drop(columns=['companyname'], axis=1)
    
    company_data = pd.concat([company_data, company_data.shift(periods=periods)], axis=1)
    company_data = company_data.dropna(axis=0)
    
    X, y = company_data.drop(columns=["priceoflasttransaction"]), company_data["priceoflasttransaction"]
    
    train_X, test_X, train_y, test_y = train_test_split(
        X, y, test_size=0.3, random_state=42, shuffle=False
    )
        
    scaler = MinMaxScaler()
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.transform(test_X)

    train_X = train_X.reshape(train_X.shape[0], lag, (train_X.shape[1] // lag))
    test_X = test_X.reshape(test_X.shape[0], lag, (test_X.shape[1] // lag))
    
    model = Sequential([
        Input((train_X.shape[1], train_X.shape[2],)),
        LSTM(64, activation="relu", return_sequences=True),
        LSTM(32, activation="relu"),
        Dense(1, activation="linear")
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    
    model.fit(train_X, train_y, validation_split=0.2, epochs=4, batch_size=16, verbose=1)
    
    model.save(f"models/{company}.keras")

Epoch 1/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 3171124224.0000 - mse: 3171124224.0000 - val_loss: 9037712384.0000 - val_mse: 9037712384.0000
Epoch 2/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3119196416.0000 - mse: 3119196416.0000 - val_loss: 6819794432.0000 - val_mse: 6819794432.0000
Epoch 3/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1612743040.0000 - mse: 1612743040.0000 - val_loss: 15195876.0000 - val_mse: 15195876.0000
Epoch 4/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6656697.5000 - mse: 6656697.5000 - val_loss: 21094866.0000 - val_mse: 21094866.0000
Epoch 1/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 1763619.8750 - mse: 1763619.8750 - val_loss: 5546214.5000 - val_mse: 5546214.5000
Epoch 2/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1080929.2500 - mse: 1080929.2500 - val_loss: 25789.7832 - val_mse: 25789.7832
Epoch 3/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 11979.9424 - mse: 11979.9424 - val_loss: 3585.5066 - val_mse: 3585.5066
Epoch 4

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

def predict_next_price(company_name, current_price, lag=3):
    prices = [current_price] * lag
    
    features = np.array(prices).reshape(1, -1)
    
    scaler = np.load(f"scalers/{company_name}_scaler.npy", allow_pickle=True).item()
    model = load_model(f"models/{company_name}.keras")
    
    features_scaled = scaler.transform(features)
    features_reshaped = features_scaled.reshape(1, lag, features_scaled.shape[1] // lag)
    
    prediction = model.predict(features_reshaped, verbose=0)
    return prediction[0, 0]

price = 400.0
predicted = predict_next_price("BANA", price)
print(f"Predicted price: {predicted:.2f}")